In [ ]:
apikey = 1K4gtNhIYa7atuvsQCNHB9NbVJvfiy8NFvJsveMl
document = https://api.regulations.gov/v4/documents?filter[docketId]=COLC-2023-0006&api_key=1K4gtNhIYa7atuvsQCNHB9NbVJvfiy8NFvJsveMl
objectId1 = 0900006485f112d2
commentlist = https://api.regulations.gov/v4/comments?filter[commentOnId]=0900006485f112d2&api_key=1K4gtNhIYa7atuvsQCNHB9NbVJvfiy8NFvJsveMl
###below works for the first 1-20 pages
commentPage1_first5000 = https://api.regulations.gov/v4/comments?filter[commentOnId]=0900006485f112d2&page[size]=250&page[number]=1&sort=lastModifiedDate,documentId&api_key=1K4gtNhIYa7atuvsQCNHB9NbVJvfiy8NFvJsveMl
commentPage2_second5000 = https://api.regulations.gov/v4/comments?filter[commentOnId]=0900006485f112d2&filter[lastModifiedDate][ge]=2023-11-01 12:57:16&page[size]=250&page[number]=1&sort=lastModifiedDate,documentId&api_key=1K4gtNhIYa7atuvsQCNHB9NbVJvfiy8NFvJsveMl
### There are two extras here
lastCommentPage_last372 = https://api.regulations.gov/v4/comments?filter[commentOnId]=0900006485f112d2&filter[lastModifiedDate][ge]=2023-12-07 10:21:24&page[size]=250&page[number]=1&sort=lastModifiedDate,documentId&api_key=1K4gtNhIYa7atuvsQCNHB9NbVJvfiy8NFvJsveMl

sampleComment = https://api.regulations.gov/v4/comments/COLC-2023-0006-0719?include=attachments&api_key=1K4gtNhIYa7atuvsQCNHB9NbVJvfiy8NFvJsveMl


In [5]:
import requests
import json
import jmespath
import csv
import pymupdf
import requests
import io
import fitz
from urllib.request import Request, urlopen
import pypdf


api_key = "1K4gtNhIYa7atuvsQCNHB9NbVJvfiy8NFvJsveMl"
doc_id = "0900006485f112d2"
page_size = 250
total_pages = 20

In [46]:
all_data = []

for page in range(1, total_pages + 1):
    url1 = f'https://api.regulations.gov/v4/comments?filter[commentOnId]={doc_id}&page[size]={page_size}&page[number]={page}&sort=lastModifiedDate,documentId&api_key={api_key}'
    url2 = f'https://api.regulations.gov/v4/comments?filter[commentOnId]={doc_id}&filter[lastModifiedDate][ge]=2023-11-01 12:57:16&page[size]=250&page[number]={page}&sort=lastModifiedDate,documentId&api_key={api_key}'
    response = requests.get(url1)
    response2 = requests.get(url2)
    if response.status_code == 200:
        data = response.json()
        all_data.extend(data['data'])
    else:
        print(f"Failed to retrieve data for page {page}")
    if response2.status_code == 200:
        data = response2.json()
        all_data.extend(data['data'])
    else:
        print(f"Failed to retrieve data for page {page}")

url3 = f'https://api.regulations.gov/v4/comments?filter[commentOnId]={doc_id}&filter[lastModifiedDate][ge]=2023-12-07 10:21:24&page[size]=250&page[number]=1&sort=lastModifiedDate,documentId&api_key={api_key}'
url4 = f'https://api.regulations.gov/v4/comments?filter[commentOnId]={doc_id}&filter[lastModifiedDate][ge]=2023-12-07 10:21:24&page[size]=250&page[number]=2&sort=lastModifiedDate,documentId&api_key={api_key}'
response = requests.get(url3)
response2 = requests.get(url4)
if response.status_code == 200:
    data = response.json()
    all_data.extend(data['data'])
if response2.status_code == 200:
    data = response2.json()
    all_data.extend(data['data'])

# Now `all_data` contains all the comments from page 1 to 20
print(f"Total comments retrieved: {len(all_data)}")
ids = [item['id'] for item in all_data]
print(ids)

with open('comment_ids.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['id'])
    for id in ids:
        writer.writerow([id])

csvfile.close()

KeyboardInterrupt: 

In [45]:
comments_data = []

def reset_eof_of_pdf_return_stream(pdf_stream_in:list):
    for i, x in enumerate(txt[::-1]):
        if b'%%EOF' in x:
            actual_line = len(pdf_stream_in)-i
            print(f'EOF found at line position {-i} = actual {actual_line}, with value {x}')
            break
    return pdf_stream_in[:actual_line]

with open('comment_ids.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip header row
    for row in reader:
        comment_id = row[0]
        comment_url = f'https://api.regulations.gov/v4/comments/{comment_id}?include=attachments&api_key={api_key}'
        response = requests.get(comment_url)
        if response.status_code == 200:
            comment_data = response.json()
            attributes = comment_data['data']['attributes']
            try:
                attachment_url = comment_data['included'][0]['attributes']['fileFormats'][0]['fileUrl']
                writer = PdfWriter()

                remoteFile = urlopen(Request(url)).read()
                memoryFile = BytesIO(remoteFile)
                pdfFile = PdfReader(memoryFile)

                text = ''
                cleanText = ''

                for pageNum in range(len(pdfFile.pages)):
                        currentPage = pdfFile.pages[pageNum]
                        text += currentPage.extract_text()
                
                comments_data.append({
                'comment': attributes.get('comment', ''),
                'attachments': text,
                'title': attributes.get('title', ''),
                'postedDate': attributes.get('postedDate', '')
                })
            except KeyError:
                comments_data.append({
                'comment': attributes.get('comment', ''),
                'attachments': [],
                'title': attributes.get('title', ''),
                'postedDate': attributes.get('postedDate', '')
                })


with open('comments_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['comment', 'attachments', 'title', 'postedDate']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(comments_data)

In [43]:
from urllib.request import Request, urlopen
from pypdf import PdfWriter, PdfReader
from io import BytesIO

url = "https://downloads.regulations.gov/COLC-2023-0006-0436/attachment_1.pdf"
writer = PdfWriter()

remoteFile = urlopen(Request(url)).read()
memoryFile = BytesIO(remoteFile)
pdfFile = PdfReader(memoryFile)

text = ''
cleanText = ''

for pageNum in range(len(pdfFile.pages)):
        currentPage = pdfFile.pages[pageNum]
        text += currentPage.extract_text()
        

print(text)

UnitedStatesCopyrightOfficeRequestforpubliccommentsontheuseofArtificialIntelligenceSystemsinthecreationofintellectualproperty.CommentPeriodopeningAugust30,2023
From:
JohnRothrockDenver,ColoradoAugust30,2023
Towhomitmayconcern,
PleasereviewthisletterinyourconsiderationofchangesandadditionstoCopyrightlawconsideringtheuseofArtificialIntelligenceSystemsinthecreationofintellectualproperty.
Mybackgroundasa“RightsHolder”IwillgetintomycommentsandsuggestionsconcerningtheinclusionintothecopyrightlawofArtificialIntelligenceSystemsusedinthecreationofintellectualpropertyinthenextsectionsofthisletter.IfeelagoodsummaryofmyexperienceasacreatorisnecessaryandvalidatesmycontributiontothepubliccommentsrequestedbytheCopyrightofficeonthismatter.So,Iwouldliketotellthatstoryfirst.
Iamacomposer,andauthor,whohascreatedover200musicalcompositions,manywithaccompanyinglyrics.Ialsohavecompletedseveralliteraryworks.IpublishmyworksunderthenamesJohnRock,RocRothrock,andJohn“Roc”Rothrock.Ihaveover45publishedmusicalworks,

In [ ]:
with open('comments.csv', 'w', newline='') as csvfile:
    fieldnames = ['Name of commenter', 'comment', 'date']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(data)